Conduct customer segmentation and market basket analysis on online retail transaction data

- dataset from kaggle
- reference: https://www.kaggle.com/mgmarques/customer-segmentation-and-market-basket-analysis/notebook

# Import package

In [1]:
# import package

import pandas as pd
import numpy as np

import datetime
import math

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
sns.set(style='ticks', font_scale=1.5)
from mpl_toolkits.mplot3d import Axes3D
import plotly as py

from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox

from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples


# Read/ Explore/ Clean data

In [2]:
# read dataset

df = pd.read_excel('Online Retail.xlsx')

In [3]:
# Take a first look of data

def rstr(df, pred=None): 
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()])
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0])
    missing_ration = (df.isnull().sum()/ obs) * 100
    skewness = df.skew()
    kurtosis = df.kurt() 
    print('Data shape:', df.shape)
    
    if pred is None:
        cols = ['types', 'counts', 'distincts', 'nulls', 'missing ration', 'uniques', 'skewness', 'kurtosis']
        str = pd.concat([types, counts, distincts, nulls, missing_ration, uniques, skewness, kurtosis], axis = 1, sort=True)

    else:
        corr = df.corr()[pred]
        str = pd.concat([types, counts, distincts, nulls, missing_ration, uniques, skewness, kurtosis, corr], axis = 1, sort=True)
        corr_col = 'corr '  + pred
        cols = ['types', 'counts', 'distincts', 'nulls', 'missing ration', 'uniques', 'skewness', 'kurtosis', corr_col ]
    
    str.columns = cols
    dtypes = str.types.value_counts()
    print('___________________________\nData types:\n',str.types.value_counts())
    print('___________________________')
    return str

details = rstr(df)
display(details.sort_values(by='missing ration', ascending=False))

df.describe()

Data shape: (541909, 8)
___________________________
Data types:
 object            4
float64           2
datetime64[ns]    1
int64             1
Name: types, dtype: int64
___________________________


,types,counts,distincts,nulls,missing ration,uniques,skewness,kurtosis
CustomerID,float64,406829,4373,135080,24.926694,"[[17850.0, 13047.0, 12583.0, 13748.0, 15100.0,...",0.029835,-1.179982
Description,object,540455,4224,1454,0.268311,"[[WHITE HANGING HEART T-LIGHT HOLDER, WHITE ME...",NaN,NaN
Country,object,541909,38,0,0.000000,"[[United Kingdom, France, Australia, Netherlan...",NaN,NaN
InvoiceDate,datetime64[ns],541909,23260,0,0.000000,"[[2010-12-01 08:26:00, 2010-12-01 08:28:00, 20...",NaN,NaN
InvoiceNo,object,541909,25900,0,0.000000,"[[536365, 536366, 536367, 536368, 536369, 5363...",NaN,NaN
Quantity,int64,541909,722,0,0.000000,"[[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 8...",-0.264076,119769.160031
StockCode,object,541909,4070,0,0.000000,"[[85123A, 71053, 84406B, 84029G, 84029E, 22752...",NaN,NaN
UnitPrice,float64,541909,1630,0,0.000000,"[[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2....",186.506972,59005.719097


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [4]:
# check the negative price and unit

print('Check if we had negative quantity and prices at same register:',
     'No' if df[(df.Quantity<0) & (df.UnitPrice<0)].shape[0] == 0 else 'Yes')
print('\nCheck how many register we have where quantity is negative',
      'and prices is 0 or vice-versa:',
      df[(df.Quantity<=0) & (df.UnitPrice<=0)].shape[0])
print('\nWhat is the customer ID of the registers above:',
      df.loc[(df.Quantity<=0) & (df.UnitPrice<=0), 
                ['CustomerID']].CustomerID.unique())
print('\n% Negative Quantity: {:3.2%}'.format(df[(df.Quantity<0)].shape[0]/df.shape[0]))
print('\nAll register with negative quantity has Invoice start with:', 
      df.loc[(df.Quantity<0) & ~(df.CustomerID.isnull()), 'InvoiceNo'].apply(lambda x: x[0]).unique())


Check if we had negative quantity and prices at same register: No

Check how many register we have where quantity is negative and prices is 0 or vice-versa: 1336

What is the customer ID of the registers above: [nan]

% Negative Quantity: 1.96%

All register with negative quantity has Invoice start with: ['C']


In [5]:
# check the negative price and unit

print('Check register with UnitPrice negative:')
display(df[(df.UnitPrice<0)])

print("\nSales records with Customer ID and zero in Unit Price:",df[(df.UnitPrice==0) & ~(df.CustomerID.isnull())].shape[0])
df[(df.UnitPrice==0)  & ~(df.CustomerID.isnull())]


Check register with UnitPrice negative:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom



Sales records with Customer ID and zero in Unit Price: 40


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9302,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647.0,Germany
33576,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,0.0,16560.0,United Kingdom
40089,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911.0,EIRE
47068,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
47070,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
56674,541109,22168,ORGANISER WOOD ANTIQUE WHITE,1,2011-01-13 15:10:00,0.0,15107.0,United Kingdom
86789,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.0,17560.0,United Kingdom
130188,547417,22062,CERAMIC BOWL WITH LOVE HEART DESIGN,36,2011-03-23 10:25:00,0.0,13239.0,United Kingdom
139453,548318,22055,MINI CAKE STAND HANGING STRAWBERY,5,2011-03-30 12:45:00,0.0,13113.0,United Kingdom
145208,548871,22162,HEART GARLAND RUSTIC PADDED,2,2011-04-04 14:42:00,0.0,14410.0,United Kingdom


In [6]:
# Remove data (without customerid, negative or return transaction)

df = df[~(df.CustomerID.isnull())]
df = df[~(df.Quantity<0)]
df = df[df.UnitPrice>0]

details = rstr(df)
details.sort_values(by='distincts', ascending=False)

Data shape: (397884, 8)
___________________________
Data types:
 object            4
float64           2
datetime64[ns]    1
int64             1
Name: types, dtype: int64
___________________________


,types,counts,distincts,nulls,missing ration,uniques,skewness,kurtosis
InvoiceNo,object,397884,18532,0,0.0,"[[536365, 536366, 536367, 536368, 536369, 5363...",-0.178524,-1.200748
InvoiceDate,datetime64[ns],397884,17282,0,0.0,"[[2010-12-01 08:26:00, 2010-12-01 08:28:00, 20...",NaN,NaN
CustomerID,float64,397884,4338,0,0.0,"[[17850.0, 13047.0, 12583.0, 13748.0, 15100.0,...",0.025729,-1.180822
Description,object,397884,3877,0,0.0,"[[WHITE HANGING HEART T-LIGHT HOLDER, WHITE ME...",NaN,NaN
StockCode,object,397884,3665,0,0.0,"[[85123A, 71053, 84406B, 84029G, 84029E, 22752...",NaN,NaN
UnitPrice,float64,397884,440,0,0.0,"[[2.55, 3.39, 2.75, 7.65, 4.25, 1.85, 1.69, 2....",204.032727,58140.396673
Quantity,int64,397884,301,0,0.0,"[[6, 8, 2, 32, 3, 4, 24, 12, 48, 18, 20, 36, 8...",409.892972,178186.243253
Country,object,397884,37,0,0.0,"[[United Kingdom, France, Australia, Netherlan...",NaN,NaN


# Customer Segmentation

## Build RMF Model

### Recency

In [7]:
reference_date = df.Invoice_Date.max() + dfatetime.timedelta(days = 1)
print('Reference Date:', reference_date)
df['days_since_last_purchase'] = (reference_date - df.Invoice_Date).astype('timedelta64[D]')
customer_history_df = df[['CustomerID','days_since_last_purchase']].groupby('CustomerID').min().reset_index()
customer_history_df.rename(columns={'days_since_last_purchase':'recency'}, inplace=True)
customer_history_df.describe().T

AttributeError: 'DataFrame' object has no attribute 'Invoice_Date'

### Fequency

In [ ]:
customer_freq = (df[['CustomerID','InvoiceNo']].groupby(['CustomerId','InvoiceNo']).customer_freqount().reset_index()).groupby(['CustomerID']).count().reset_index()
customer_freq.rename(columns={'InvoiceNo':'frequency'},inplace=True)
customer_history_df = customer_history_df.merge(customer_freq)

### Monetary 

In [ ]:
customer_monetary_val = df[['Customer','amount']].groupby('CustomerID').sum().reset_index()
customer_history_df = customer_history_df.merge(customer_monetary_val)



## Data Preprocessing

In [ ]:
customer_history_df['recency_log'] = customer_history_df['recency'].apply(math.log)
customer_history_df['frequency_log'] = customer_history_df['frequency'].apply(math.log)
customer_history_df['amount_log'] = customer_history_df['amount'].apply(math.log)
feature_vector = ['amount_log','recency_log','frequency_log']
X_subset  =customer_history_df[feature_vector]
scaler = Processing.StandardScaler().fit(X_subset)
X_scaled = scaler.transform(X_subset)
pd.DataFrame(X_scaled, columns=subset.columns).describe().T


## Clustering

In [ ]:
SSE=[]

for cluster in range(1,10):
    model=KMeans(n_clusters=cluster)
    model.fit(customer_history_df)
    SSE.append(model.inertia_)

plt.figure(figsize=(12,6))
plt.plot(range(1,10),SSE, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.title('Elbow plot')
plt.show()

# Silhouette coefficient
model = KMeans(n_clusters=4)
model.fit(customer_history_df)
print(silhouette_score(customer_history_df, model.labels_))

In [ ]:
# look at the cluster center values after returning them to normal values from the log and scaled version
features = ['amount',  'recency',  'frequency']
for i in range(3,K_best+1,2):
    print("for {} clusters the silhouette score is {:1.2f}".format(i, cluster_centers[i]['silhouette_score']))
    print("Centers of each cluster:")
    cent_transformed = scaler.inverse_transform(cluster_centers[i]['cluster_center'])
    print(pd.DataFrame(np.exp(cent_transformed),columns=features))
    print('-'*50)

# Cross Selling

## Build Transaction Dataset

In [ ]:
items = list(df.Description.unique())
grouped = df.groupby('InvoiceNo')
transaction_level = grouped.aggregate(lambda x : tuple(x)).reset_index()[['InvoiceNo','Description']]
transaction_dict = {item:0 for item in items}
output_dict = dict()
temp = dict()

for rec in transaction_level.to_dict('records'):
    invoice_num = rec['InvoiceNo']
    items_list = rec['Description']
    transaction_dict = {item:0 foror item in items}
    transaction_dict.update({item:1 for item in items if item in items_list})
    temp.update({invoice_num:transaction_dict})

new = [v for k,v in temp.items()]
transaction_df = pd.DataFrame(new)



## Prune Dataset for Frequently Purchased Items

In [ ]:
def prune_dataset(input_df, length_trans = 2, total_sales_perc = 0.5, 
                  start_item = None, end_item = None, TopCols = None):
    if 'total_items' in input_df.columns:
        del(input_df['total_items'])
    item_count = input_df.sum().sort_values(ascending = False).reset_index()
    total_items = sum(input_df.sum().sort_values(ascending = False))
    item_count.rename(columns={item_count.columns[0]:'item_name',
                               item_count.columns[1]:'item_count'}, inplace=True)
    if TopCols: 
        input_df['total_items'] = input_df[TopCols].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[TopCols], item_count[item_count.item_name.isin(TopCols)]
    elif end_item > start_item:
        selected_items = list(item_count[start_item:end_item].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items],item_count[start_item:end_item]
    else:
        item_count['item_perc'] = item_count['item_count']/total_items
        item_count['total_perc'] = item_count.item_perc.cumsum()
        selected_items = list(item_count[item_count.total_perc < total_sales_perc].item_name)
        input_df['total_items'] = input_df[selected_items].sum(axis = 1)
        input_df = input_df[input_df.total_items >= length_trans]
        del(input_df['total_items'])
        return input_df[selected_items], item_count[item_count.total_perc < total_sales_perc]

In [ ]:
output_df, item_counts = prune_dataset(input_df=transaction_df, length_trans=2,start_item = 0, end_item = 15)
print('Total of Sales Amount by the Top 15 Products in Sales Events (Invoice): {:.2f}'.format(AmoutSum[Top15ev].sum()))
print('Number of Sales Events:', output_df.shape[0])
print('Number of Products:', output_df.shape[1])

item_counts

## Association Rule Mining with FP Growth